In [1]:
!pip install google-cloud-storage pandas


In [17]:
import zstandard as zstd
import os
import subprocess
import pandas as pd


In [18]:

gsutil_path = 'gs://cloud-samples-data/bigquery/sample-transactions/transactions.csv'
local_path = '/tmp/transactions.csv'

df = pd.read_csv(local_path)



In [19]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [20]:
filtered_transactions = df[df['transaction_amount'] > 20].copy()

In [21]:
filtered_transactions = filtered_transactions[filtered_transactions['timestamp'].dt.year >= 2010].copy()

In [27]:
filtered_transactions['date'] = filtered_transactions['timestamp'].dt.date
sum_by_date = filtered_transactions.groupby('date')['transaction_amount'].sum().reset_index()

In [28]:
sum_by_date.to_csv(index=False)


'date,transaction_amount\n2017-03-18,2102.22\n2017-08-31,13700000023.08\n2018-02-27,129.12\n'

In [29]:
json_str = sum_by_date.to_json(orient='records', lines=True)

output_dir = 'output'
os.makedirs(output_dir, exist_ok=True)
json_file_path = os.path.join(output_dir, 'results.jsonl.z')

with open(json_file_path, 'wb') as json_file:
    compressor = zstd.ZstdCompressor()
    compressed = compressor.compress(json_str.encode('utf-8'))
    json_file.write(compressed)


In [30]:
gitignore_path = '.gitignore'
with open(gitignore_path, 'a') as gitignore_file:
    gitignore_file.write('\n# Ignore all files in the output directory\noutput/*\n')